In [ ]:
import pickle
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression

In [ ]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

In [ ]:
# Attention, pour la régression logistique, les images doivent être applaties (flattened) en dimension 1
N = len(noms)

visages = visages.reshape(N, -1)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(visages, noms)

In [ ]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour une caméra interne, 1 pour caméra externe
user = {}
test = "\n"
setNom = list(set(noms))
for i in range(len(setNom)):
    user[i] = setNom[i]
    test += str(i) + "." + setNom[i] + "\n"

In [ ]:
# Demande à l'utilisateur de saisir les numéros des utilisateurs admis
numAdmin = list(map(int, input("Qui sont admis : " + test).split()))

# Crée une liste vide pour stocker les noms des administrateurs
admin = []

# Parcours la liste des numéros des utilisateurs admis
for num in numAdmin:
    # Ajoute le nom correspondant au numéro à la liste des administrateurs
    admin.append(user[num])

# Boucle principale pour la reconnaissance faciale
while True:
    # Capture une trame depuis la caméra
    ret, trame = camera.read()

    # Vérifie si la capture est réussie
    if ret == True:
        # Convertit l'image en niveaux de gris
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        
        # Détecte les visages dans l'image en niveaux de gris
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        # Parcours chaque visage détecté
        for (x, y, l, h) in coordonnees_visage:
            # Extrait le visage de la trame
            visage = trame[y:y + h, x:x + l, :]
            
            # Redimensionne le visage à 50x50 pixels et le transforme en vecteur
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1, -1)
            
            # Prédit le nom du visage à l'aide du modèle de régression logistique (non défini ici)
            texte = log_reg.predict(visage_redimensionne)
            
            # Vérifie si le nom prédit est dans la liste des administrateurs
            if texte[0] in admin:
                data = texte[0] + " admis"
                col = (0, 255, 0)  # Couleur verte pour les utilisateurs admis
            else:
                col = (255, 0, 0)  # Couleur rouge pour les utilisateurs non admis
                data = texte[0] + " non admis"
            
            # Affiche le statut de l'utilisateur au-dessus du visage détecté
            cv2.putText(trame, data, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            
            # Dessine un rectangle autour du visage détecté
            cv2.rectangle(trame, (x, y), (x + l, y + h), col, 2)

        # Affiche l'image avec les rectangles et les noms
        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        # Vérifie si la touche 'Esc' est pressée pour quitter
        if cv2.waitKey(1) == 27:
            break
            
    else:
        # Affiche une erreur si la capture échoue
        print("erreur")
        break

# Ferme toutes les fenêtres OpenCV
cv2.destroyAllWindows()

# Libère la ressource de la caméra
camera.release()
